In [4]:
import requests
import os
from requests.compat import urljoin
from bs4 import BeautifulSoup

caminho_base = '/content/drive/MyDrive/Aula_Web_Mining'

def links_validos(href):
  if href != None and href != '' and href[0] != "/wiki/":
    return True
  return False

def busca_links(conteudo, link):
  # criando objeto do BS 
  bsObj = BeautifulSoup(conteudo, 'html.parser')
  content = bsObj(id='content')[0]
  # Encontro marcados de link (a)
  links = content('a', attrs={'href':links_validos})
  # Extraindo o atributo hred dos links, juntando com o endereço principal
  links = [urljoin(link, l.get('href')) for l in links]
  return links

def salva_arquivo(pasta, nome, conteudo):

  if not os.path.isdir(pasta):
    os.makedirs(pasta, exist_ok=True)

  with open(f'{pasta}/{nome}', 'w', encoding='utf8') as arq:
    # bloco de execução enquanto o arquivo está aberto
    arq.write(conteudo)

def crawler(buscar, buscados, stop = 1):
  if buscar and len(buscados) < stop:
    link = buscar.pop()
    if link not in buscados:
      print(f'Baixando {link}')
      response = requests.get(link)
      html = response.text
            
      bsObj = BeautifulSoup(html, 'html.parser')
            
      for r in bsObj(['script', 'style']): r.extract()
      content = bsObj(id='content')[0]
            
      salva_arquivo(
          pasta=f'{caminho_base}/wiki2', 
          nome=f'wiki2{len(buscados)+1}.txt', 
          conteudo=response.text
      )
      buscados.add(link)
      novos_links = busca_links(response.text, link)
      buscar.update(novos_links)
    crawler(buscar, buscados, stop) 

init = 'https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial'
buscar = [init]
buscados = []
crawler(set(buscar), set(buscados),100)


Baixando https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial
Baixando https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial#Inteligência_artificial_forte
Baixando https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial#cite_ref-its2_56-0
Baixando https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial#Referências
Baixando https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial#cite_note-36
Baixando https://pt.wikipedia.org/wiki/Especial:Fontes_de_livros/85-346-0122-4
Baixando https://pt.wikipedia.org/wiki/Especial:Fontes_de_livros/85-346-0122-4#Timor-Leste
Baixando https://pt.wikipedia.org/wiki/Macsyma
Baixando https://pt.wikipedia.org/w/index.php?title=Intelig%C3%AAncia_artificial&veaction=edit&section=23
Baixando https://pt.wikipedia.org/wiki/Qu%C3%ADmica_verde
Baixando https://pt.wikipedia.org/wiki/Especial:Fontes_de_livros/85-346-0122-4#Portugal
Baixando https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial#Inteligência_Artificial_eliminará_todos_os_cargos

IndexError: ignored